In [2]:
# %cd drive/MyDrive/Colab\ Notebooks/COMP8420

/content/drive/MyDrive/Colab Notebooks/COMP8420


This notebook is for creating result for baseline model

# Import modules

In [3]:
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
from pandas import DataFrame

In [4]:
import os
from typing import Tuple
from tqdm.notebook import tqdm

# Load data function

In [5]:
def load_data(path:str, mode:str)-> DataFrame:
    """This function load data from specific data file based on mode

    Args:
        path (str): path to folder contains data file
        mode (str): what data to be loaded

    Returns:
        DataFrame: DataFrame contains all data
    """
    assert mode in ["gsr", "pupil", "skin", "all"], f"mode should be 'gsr','pupil','all'. Not {mode}"
    # read data gsr
    if mode in ['gsr', 'all']:
        df_gsr = pd.read_excel(path+'gsr_features.xlsx',sheet_name='data')
        df_gsr.columns = ['patient'] + list(df_gsr.columns)[1:]
        if mode == 'gsr':
            return df_gsr
    # read pupil data
    if mode in ['pupil', 'all']:
        df_pupil = pd.read_excel(path+'pupil_features.xlsx',sheet_name='data')
        df_pupil.columns = ['patient'] + list(df_pupil.columns)[1:]
        if mode == 'pupil':
            return df_pupil
    # read skin temperature data
    if mode in ['skin', 'all']:
        df_skin = pd.read_excel(path+'skintemp_features.xlsx',sheet_name='data')
        df_skin.columns = ['patient'] + list(df_skin.columns)[1:]
        if mode == 'skin':
            return df_skin
    # in case of using all data, combine
    data = pd.concat([df_gsr,df_pupil,df_skin],axis = 1)
    # remove duplicated columns
    data = data.loc[:,~data.columns.duplicated()]
    return data

def train_val_test_split(data:DataFrame, test_patient:str, val_patient:str) -> Tuple[DataFrame,DataFrame,DataFrame]:
    """ This function is to create train, test and validation dataset using leave-one-patient-out

    Args:
        data (DataFrame): the dataset that contains all data point
        test_patient (str): patient choosen for testing
        val_patient (str): patient choosen for validation in training process

    Returns:
        Tuple[DataFrame,DataFrame,DataFrame]: train, validation, test dataframe
    """
    # check input
    assert test_patient in data.patient.unique(), f'{data.patient.unique().tolist()}'
    assert val_patient in data.patient.unique(), f'{data.patient.unique().tolist()}'
    # create train and test dataset
    df_test = data[data['patient'] == test_patient]
    df_train = data[(data['patient'] != test_patient)&((data['patient'] != val_patient))]
    df_val = data[data['patient'] == val_patient]
    # reset index
    df_test = df_test.reset_index(drop=True)
    df_train = df_train.reset_index(drop=True)
    df_val = df_val.reset_index(drop = True)
    # delete patient column
    if 'patient' in df_train.columns:
        del df_train['patient']
    if 'patient' in df_test.columns:
        del df_test['patient']  
    if 'patient' in df_val.columns:
        del df_val['patient']
    return df_train, df_val, df_test


def df_to_xy(df:DataFrame) -> Tuple[np.ndarray, np.ndarray]:
    """Convert dataframe to X and Y numpy array where X is input and Y is output

    Args:
        df (DataFrame): dataset to convert to numpy array

    Returns:
        Tuple[np.ndarray, np.ndarray]: X,Y
    """
    df = df.apply(pd.to_numeric)
    x = df.values[:,1:]
    y = df.values[:,0]
    return x,y

# Model

In [6]:
class BaselineNN(nn.Module):
    """Baseline Neural Network with 1 hidden layer
    """
    def __init__(self, input_size: int, hidden_size:int, output_size:int, p = 0.7):
        """
        Args:
            input_size (int): input size of input vector
            hidden_size (int): size of hidden layer
            output_size (int): size of output layer
            p (float): probability of Dropout
        """
        super().__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        # from input to hidden layer
        self.in2hid = nn.Linear(self.input_size,self.hidden_size, bias=True)
        # from hidden to output layer
        self.hid2out = nn.Linear(self.hidden_size, self.output_size, bias=True)
        # dropout layer to generalize
        self.dropout = nn.Dropout(p)

    def forward(self, input):
        x = self.in2hid(input)
        x = torch.sigmoid(x)
        x = self.dropout(x)
        output = self.hid2out(x)
        return output

# Ultities

In [7]:
def accuracy(Y_pred, Y_target):
    """ Calculate accuracy of prediction (Y_pred) compared to truth label (Y_target)
    Args:
        Y_pred (torch.tensor): prediction tensor with size of [batch x num_class]
        Y_target (torch.tensor): truth tensor with size of [batch x 1]
    Returns:
        torch.float: the overall accuracy
    """
    # find number of correct label
    correct_num = (Y_pred.argmax(-1) == Y_target).int().sum()
    # calculate accuracy
    acc = correct_num.float()/Y_target.shape[0]
    return acc


def recall(Y_pred, Y_target, class_label):
    """ Calculate Recall
    Recall = TP/(TP+FN)
    Args:
        Y_pred (torch.tensor): prediction tensor with size of [batch x num_class]
        Y_target (torch.tensor): truth tensor with size of [batch x 1]
        claass_label (str): label of class that we want to calculate recall
    Returns:
        torch.float: recall of model
    """
    # get label prediction
    Y_pred = Y_pred.argmax(-1)
    # find all datapoint that is true for the class_label
    label_index = torch.where(Y_target == class_label)
    # find all correctly predicted label in this class
    correct_label = (Y_pred[label_index] == Y_target[label_index]).int().sum()
    # returnr recall
    return float(correct_label)/len(label_index[0])


def precision(Y_pred, Y_target, class_label):
    """ Calculate Precision
    Recall = TP/(TP+FP)
    Args:
        Y_pred (torch.tensor): prediction tensor with size of [batch x num_class]
        Y_target (torch.tensor): truth tensor with size of [batch x 1]
        claass_label (str): label of class that we want to calculate precision
    Returns:
        torch.float: precision of model
    """
    # get label prediction
    Y_pred = Y_pred.argmax(-1)
    # get all correct label index
    label_index = torch.where(Y_target == class_label)
    # get all correctly predicted label for the class label
    correct_label = (Y_pred[label_index] == Y_target[label_index]).int().sum()
    # return precision
    return float(correct_label)/max(len(torch.where(Y_pred == class_label)[0]),1)


def f1(recall, precision):
    """ Calculate F1 score
    F1 =  2 * precision * recall/(precision + recall)
    Args:
        recall (float): recall value
        precision (float): precision value
    Returns:
        float: f1 score
    """
    return 2 * recall * precision / max((recall + precision),1)

In [8]:
def train_model(X_train, Y_train, num_epoch, print_freq):
    """
    This function is used to train baseline model
    Args:
        X_train (torch.tensor): input tensor used for training with size of [batch x dimension]
        Y_train (torch.tensor): output tensor used for training with size of [batch x 1]
        num_epoch (int): number of training epoch
        print_freq (int or bool): print process for debugging, if don't need to, set it to False
    Returns:
        Tuple(mode, loss_function): return trained model and loss function
    """
    # get input size from input variable
    input_size = X_train.shape[1]
    # output size
    output_size = 4
    # hidden layer size
    hidden_size = 50
    # create model
    model = BaselineNN(input_size, hidden_size, output_size)
    # create loss function
    criterion = nn.CrossEntropyLoss()
    # use Adam as optimizer
    optimizer = optim.Adam(model.parameters(), lr=0.01)

    # for epoch in tqdm(range(num_epoch)):
    for epoch in range(num_epoch):
        # set model to train mode
        model.train()
        optimizer.zero_grad()

        outputs = model(X_train)
        train_loss = criterion(outputs, Y_train)
        # calculate accuracy if need to print out
        train_acc = (outputs.argmax(-1) == Y_train).int().sum()/X_train.shape[0]

        train_loss.backward()
        optimizer.step()
        # when print_preq != False
        if print_freq:
            if epoch % print_freq == 0:
                print(f'[EPOCH {epoch+1}/{num_epoch}] train_loss {train_loss.item():.4f}\t train_acc {train_acc:.4f}')
                # pass
    return model, criterion

def validation(model, criterion, X_val, Y_val, test = False):
    """ Calculate all metrics of trained model
    Args:
        model (nn.Module): the model we need to evaluate
        criterion (nn.Module): loss function
        X_val (torch.tensor): input tensor used for validation with size of [batch x dimension]
        Y_val (torch.tensor): output tensor used for validation with size of [batch x 1]
        test (bool): set this to print output to debug/compare
    Returns:
        loss: loss function value
        acc: accuracy of model
        rc_all: recall for each class
        pr_all: precision for each class
        f1_all: F1 score for each class
    """
    # set model to evaluation mode
    model.eval()
    # calculate output
    outputs = model(X_val)
    # calculate loss_function
    loss = criterion(outputs, Y_val)
    # calculate accuracy
    acc = accuracy(outputs, Y_val)

    rc_all = []
    pr_all = []
    f1_all = []
    for i in range(4):
        # recall
        rc = recall(outputs, Y_val, i)
        # precision
        pr = precision(outputs, Y_val, i)
        # f1
        f1_value = f1(rc,pr)
        # Store result
        rc_all.append(rc)
        pr_all.append(pr)
        f1_all.append(f1_value)
    if test:
        print('Predict\t',outputs.argmax(-1))
        print('Test\t',Y_val)
    return loss, acc, rc_all, pr_all, f1_all

In [10]:
# 12321
torch.manual_seed(87651)
# 268
np.random.seed(268)
# data folder
root = './data/'
# load data set
dataset = load_data(root, 'all')
# get all the patient to perform leave one patient out
patient_list = dataset.patient.unique().tolist()
# store metrics data
all_acc = []
all_recall = []
all_precision = []
all_f1 = []

for patient in patient_list:
    print(f'Choose patient {patient} as testing patient')
    # choose accuracy to save best model
    best_accuracy = 0
    best_p_val = None
    # start cross validation
    for val_p in patient_list:
        if val_p == patient:
            continue
        print(f'Choose patient {val_p} as validation patient')
        # get train, val, test data
        train, val, test = train_val_test_split(dataset, patient,val_p)
        # create x,y
        train_x, train_y = df_to_xy(train)
        val_x, val_y = df_to_xy(val)
        # calculate normalize parameter
        mean_train_x = train_x.mean(axis=0)
        std_train_x = train_x.std(axis=0)
        # normalize data
        normalize_train_x = (train_x - mean_train_x)/std_train_x
        normalize_val_x = (val_x - mean_train_x)/std_train_x
        # convert to tensor
        X_train = torch.tensor(normalize_train_x, dtype=torch.float)
        Y_train = torch.tensor(train_y, dtype=torch.long)

        X_val = torch.tensor(normalize_val_x, dtype=torch.float)
        Y_val = torch.tensor(val_y, dtype=torch.long)
        # train model
        model, loss_func = train_model(X_train, Y_train, 900,False)
        # validate model
        val_loss, val_acc, rc_all, pr_all, f1_all = validation(model, loss_func, X_val,Y_val)
        # save best model
        if val_acc > best_accuracy:
            best_accuracy = val_acc
            # save model
            torch.save(model.state_dict(), './best_model/{model_name}_{patient}.pth'.format(model_name=type(model).__name__, patient=patient))
            best_p_val = val_p

    # load best model to calculate accuracy
    model.load_state_dict(torch.load('./best_model/{model_name}_{patient}.pth'.format(model_name=type(model).__name__, patient=patient)))
    model.eval()
    # get train, val, test data
    train, val, test = train_val_test_split(dataset, patient,best_p_val)
    # create x,y
    train_x, train_y = df_to_xy(train)
    test_x, test_y = df_to_xy(test)
    # calculate normalize parameter
    mean_train_x = train_x.mean(axis=0)
    std_train_x = train_x.std(axis=0)
    # normalize data
    normalize_test_x = (test_x - mean_train_x)/std_train_x
    # convert to tensor
    X_test = torch.tensor(normalize_test_x, dtype=torch.float)
    Y_test = torch.tensor(test_y, dtype=torch.long)
    # validate with test
    test_loss, test_acc, rc_all, pr_all, f1_all = validation(model, loss_func, X_test,Y_test, True)
    all_acc.append(test_acc)
    all_recall.append(rc_all)
    all_precision.append(pr_all)
    all_f1.append(f1_all)
# print out result
all_acc = np.array(all_acc)
all_recall = np.array(all_recall)
all_precision = np.array(all_precision)
all_f1 = np.array(all_f1)
print("==============================")
print('Average accuracy:',all_acc.mean())
print('Recall:',all_recall.mean(axis=0))
print('Precision:',all_precision.mean(axis=0))
print('F1:',all_f1.mean(axis=0))
print('Average Recall:',all_recall.mean(axis=0).mean())
print('Average Precision:',all_precision.mean(axis=0).mean())
print('Average F1:',all_f1.mean(axis=0).mean())

Choose patient p02 as testing patient
Choose patient p03 as validation patient
Choose patient p04 as validation patient
Choose patient p06 as validation patient
Choose patient p07 as validation patient
Choose patient p08 as validation patient
Choose patient p09 as validation patient
Choose patient p10 as validation patient
Choose patient p11 as validation patient
Choose patient p12 as validation patient
Choose patient p13 as validation patient
Choose patient p14 as validation patient
Predict	 tensor([1, 1, 3, 1, 1, 1, 1, 0, 0, 3, 2, 1, 2, 2, 0, 3])
Test	 tensor([2, 2, 2, 2, 0, 0, 0, 0, 3, 3, 3, 3, 1, 1, 1, 1])
Choose patient p03 as testing patient
Choose patient p02 as validation patient
Choose patient p04 as validation patient
Choose patient p06 as validation patient
Choose patient p07 as validation patient
Choose patient p08 as validation patient
Choose patient p09 as validation patient
Choose patient p10 as validation patient
Choose patient p11 as validation patient
Choose patient p

In [ ]:
type(model).__name__

'BaselineNN'